# Lecture 16: Boundary Value and Eigenvalue Problems 🫨

## 1-D Temperature BVP: C&C Example 27.3

$$\frac{d^2 T}{dx^2} + K(T_a - T) = 0$$

Boundary Conditions:
* $T(0)=T_0$
* $T(L)=T_{L}$

Defined Parameters:
* $K$: heat transfer coefficient = 0.01 m-2
* $T_a$: ambient temperature = 20°C
* $T_0$: left boundary = 40°C
* $T_{L}$: right boundary = 200°C
* $L$: length = 10 m

In [ ]:
# Import Libraries
import numpy as np
from matplotlib import pyplot as plt

# Define Parameters
T0 = 40 # left BC, deg C
TL = 200 # right BC, deg C
K = 0.01 # m^-2
Ta = 20 # ambient temp
L = 10 # m
n = 7 # number of internal nodes
dx = L / (n+1)

Finite Difference Method gives us:
$$-T_{i-1}+(2+K\Delta x^2)T_i-T_{i+1}=K\Delta x^2 T_a$$

### Translate the series of equations described above into matrix form ($Ax=b$) for $n=7$ internal nodes.

💪 Start by creating matrix $A$. Recall that $x$ comprises the following variables:

$$x=
\begin{bmatrix}
T_1 \\
T_2 \\
\vdots \\
T_7
\end{bmatrix}
$$

In [ ]:
# Initialize A
A = np.zeros([n,n])

# Assign Values to A (Note: There are many ways to do this)
#[implement code here]

print(A)

In [ ]:
# Alternative way to set up A
main = (2 + K*dx**2) * np.ones(n) # main diagonal
off1 = -1 * np.ones(n-1) # off-diagonals have n-1 elements
A = np.diag(main) + np.diag(off1,-1) + np.diag(off1, 1)
print(A)

* This alternative approach avoids Python-level loops, which are slower than vectorized NumPy operations.

💪 Now create vector $b$. Don't forget to implement our boundary conditions.

In [ ]:
# RHS vector
b = #[insert code here]

# Apply BCs to first and last element
b[0]  #[insert code here]
b[-1] #[insert code here]

💪 Use `np.linalg.solve(A,b)` to solve for $T_i$ for the internal nodes and concatenate with the boundary conditions ($T_0$ and $T_L$). Compare this numerical solution with the analytical solution:

$$T_{analytical}=73.4523 e^{0.1x}-53.4523 e^{-0.1x}+20$$

Plot the result of both the numerical and analytical solutions on a plot.

In [ ]:
# Solve Our System of Equations
T = #[insert code here]

# Append our BCs to T
T = np.concatenate((#[insert code here]))

# Calculate Analytical Solution
x = np.arange(#[insert code here])
T_analytical = 73.4523*np.exp(0.1*x) - 53.4523*np.exp(-0.1*x) + 20

# Plot Comparison
plt.plot(x, T)
plt.plot(x, T_analytical, 'ok')
plt.xlabel('x (m)')
plt.ylabel('T (deg C)')
plt.legend(['Finite Difference', 'Analytical'])
#plt.ylim([108.4,108.8]) # Zoom in to see how accurate the numerical solution is.
#plt.xlim([4.99,5.01])
plt.show()

## 🤝 1-D Axially loaded column BVP: C&C Example 27.5

$$\frac{d^2 y}{dx^2} + p^2 y = 0$$

Boundary Conditions: $y(0) = y(L) = 0$

Critical Buckling Frequencies: $p^2 = \frac{P}{EI}$.
* Note that $p=\frac{n\pi}{L}$ (discrete values) to satisfy boundary conditions.

Defined Parametrs:
- $P$: axial load, N
- $E$: modulus of elasticity, $10 \times 10^{9}$ Pa
- $I$: moment of inertia of the cross-section, $1.25 \times 10^{-5}$ m$^4$
- $L$: length, 3 m


In [ ]:
# Define Parameters
E = 10 * 10**9 # Pa
I = 1.25 * 10**-5 # m^4
L = 3 # m

# Let's look at a system with three internal nodes:
n = 3 # number of internal nodes
dx = L / (n+1)

Use a finite difference approach to estimate the eigenvalues of this matrix.

$$-y_{i-1}+2y_i-y_{i+1} = \Delta x^2 p^2 y_i$$

The values we find will be $\lambda = \Delta x^2 p^2$. Report the answer in terms of $p$ and compare to known values $p = \frac{n\pi}{L}, n=1,2,3...$

In [ ]:
# LHS matrix
main = (2) * np.ones(n) # main diagonal
off1 = -1 * np.ones(n-1) # off-diagonals have n-1 elements
A = np.diag(main) + np.diag(off1,-1) + np.diag(off1, 1) # assemble matrix A
print(A)

Solve for the eigenvalues of $A$ using `np.linalg.eig(A)` function (see documentation [here](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html)). These values will corresponds with $\Delta x^2 p^2$.

In [ ]:
# Find eigenvalues and eigenvectors of A
l,v = np.linalg.eig(A)
print(l)

* The eigenvalues values are not guaranteed to be sorted, so we could find the smallest value using `l.min()` instead of the last array value.
* We can get more eigenvalues, and better estimates, by increasing the size of the number of nodes $n$ in the finite difference approximation.

Solve for the lowest buckling frequency ($p$): $p=\sqrt{\lambda_1/\Delta x^2}$

In [ ]:
p = np.sqrt(l.min() / dx**2)
print(p)

From Table 27.5, the true value is $p=\pi/L = 1.0472$. This approximation has a relative error of about 2.5%. This value of $p$ leads to a critical load of $P = p^2 EI$.

In [ ]:
P_crit = p**2 * E * I
print('P_crit = ', P_crit/1000, ' kN')

Last, the eigenvectors `v` represent the normalized amplitudes (or the shape of the buckling column). They are returned as a matrix, where column $v_i$ is the eigenvector associated with eigenvalue $\lambda_i$.

In [ ]:
# Sort eigenvalues and reorder eigenvectors
idx = np.argsort(l)      # Indices that sort eigenvalues from smallest to largest
l_sorted = l[idx]        # Sorted eigenvalues
v_sorted = v[:, idx]     # Columns reordered to match sorted eigenvalues

# Add BCs (zeros at top and bottom) for each sorted eigenvector
S = np.vstack((np.zeros((1, 3)), v_sorted, np.zeros((1, 3))))

# Plot each mode shape vs vertical position
plt.plot(S, np.arange(5))  # Each column of S is a mode shape
plt.axvline(0, color='black', linestyle='--')
plt.legend([fr'$\lambda_{{{i+1}}}$ = {l_sorted[i]:.3f}' for i in range(3)])
plt.title("Buckling Mode Shapes")
plt.xlabel("Deflection")
plt.ylabel("Node Index")
plt.grid(True)
plt.show()